In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import null_space, svd, rq
import json

In [2]:
with open("cameraParams.json") as f:
    content = json.load(f)
    gabarito = np.array(content['cameraTransform']['rows'])
    gabarito2 = np.array(content['viewTransform']['rows'])

In [3]:
width, height = 2048, 1360

coords = np.array(( 
    (0,    0,    0, 860.4816753926701, 829.2356020942409), 
    (0,    0,    1, 755.9371727748690, 786.3455497382199), 
    (1,    0,    0, 981.1099476439790, 802.4293193717278), 
    (0,    1,    0, 852.4397905759163, 976.6701570680627), 
    (0,    1,    1, 753.2565445026178, 925.7382198952879), 
    (1,    1,    0, 970.3874345549738, 947.1832460732984),
    (1,    0,    1, 879.2460732984293, 759.5392670157069),
    (1,    1,    1, 879.2460732984293, 901.6125654450262),
    (0,    0,    2, 667.4764397905759, 740.7748691099475)
)).round()

In [4]:
a=[]
for i in range(coords.shape[0]): # para cada ponto e correspondente 3D
  x,y,z,u,v = coords[i,:]
  a.append([x,y,z,1,0,0,0,0,-u*x,-u*y,-u*z,-u])
  a.append([0,0,0,0,x,y,z,1,-v*x,-v*y,-v*z,-v])

In [9]:
a=np.array(a)
U, s, Vh = svd(a)
nm = Vh[-1,:]
nm /= nm [-1]

P = np.matrix(nm.reshape((3,4))) # Matrix de projeção
M = P[:,:3] # slice da matriz P

In [10]:
# ESTE É O VETOR POSIÇÃO DA CÂMERA
C = (np.linalg.inv(M) @ P[:, -1])
C.reshape(-1)

matrix([[11.66329729,  6.42106631, 14.19826186]])

In [15]:
# RQ decomposition - separar intrinsec de extrinsec
K, R = rq(M)

In [16]:
# Correção de sinais na diagonal da matriz intrínseca
T = np.diag(np.sign(np.diag(K)));
K = K @ T;
R = T @ R; 

In [17]:
# Calculo de offset para fazer o eixo coincidir com a quina da mesa
2*(K[0,2]/K[2,2] - width/2), (K[1,2]/K[2,2] - height)

(177.90418063753987, -167.2946041615212)

In [18]:
# Extrinsic completa
extrinsic = np.hstack([R, -R@C])

In [20]:
# Matrix da camera
camera_matrix = np.array(np.vstack([extrinsic, [0,0,0,1]]))
camera_matrix.reshape(-1)

array([  0.73309873,  -0.079348  ,  -0.67547772,   1.5497599 ,
        -0.24203979,   0.89771331,  -0.36814067,   2.28566293,
         0.63559657,   0.43337594,   0.63890711, -19.26725785,
         0.        ,   0.        ,   0.        ,   1.        ])

In [21]:
# Trocamos as linhas 2 e 3 de posição entre si pois as convenções usadas e a do three js são diferentes
camera_matrix[[1,2]] = camera_matrix[[2,1]] 
camera_matrix.reshape(-1) 


array([  0.73309873,  -0.079348  ,  -0.67547772,   1.5497599 ,
         0.63559657,   0.43337594,   0.63890711, -19.26725785,
        -0.24203979,   0.89771331,  -0.36814067,   2.28566293,
         0.        ,   0.        ,   0.        ,   1.        ])